In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\US593\\OneDrive\\Desktop\\deep-coccidiosis-detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\US593\\OneDrive\\Desktop\\deep-coccidiosis-detection'

In [5]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    source_local_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.utils import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            source_local_url=config.source_local_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [9]:
import os
import sys
import shutil
import urllib.request as request
from pathlib import Path
import zipfile
from src.cnnClassifier.logger import logging
from src.cnnClassifier.exception import CustomException
from src.cnnClassifier.utils.utils import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def copy_or_download_file(self):
        source_file_path = self.config.source_local_url
        try:
            if source_file_path:  # Copy the file if source path is provided
                if os.path.exists(source_file_path):
                    if not os.path.exists(self.config.local_data_file):    
                        shutil.copy2(source_file_path, self.config.local_data_file)
                        logging.info(f"File copied from {source_file_path} to {self.config.local_data_file}")
                    else:
                        logging.info(f"Local file already exists at: {self.config.local_data_file}")
                        logging.info(f"File size: {get_size(Path(self.config.local_data_file))}")
                else:
                    logging.info(f"Source file not found: {source_file_path}")
                    raise FileNotFoundError(f"Source file not found: {source_file_path}")
            else:  # Download the file if it doesn't already exist
                if not os.path.exists(self.config.local_data_file):
                    filename, headers = request.urlretrieve(
                        url=self.config.source_url,
                        filename=self.config.local_data_file
                    )
                    logging.info(f"{filename} downloaded! Info: \n{headers}")
                else:
                    logging.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        except Exception as e:
            raise CustomException(e, sys)

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        try :
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
        except Exception as e: 
            raise CustomException(e,sys)

In [10]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.copy_or_download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise CustomException(e,sys)

[2024-09-20 20:21:19,686] 24 root - INFO - yaml file : c:\Users\US593\OneDrive\Desktop\deep-coccidiosis-detection\config\config.yaml loaded successfully
[2024-09-20 20:21:19,688] 24 root - INFO - yaml file : c:\Users\US593\OneDrive\Desktop\deep-coccidiosis-detection\params.yaml loaded successfully
[2024-09-20 20:21:19,689] 50 root - INFO - directory already exists at : c:\Users\US593\OneDrive\Desktop\deep-coccidiosis-detection\artifacts
[2024-09-20 20:21:19,691] 50 root - INFO - directory already exists at : c:\Users\US593\OneDrive\Desktop\deep-coccidiosis-detection\artifacts\data_ingestion
[2024-09-20 20:21:19,818] 20 root - INFO - File copied from C:/Users/US593/Work/Datasets/Deep-Coccidiosis-Detection.zip to artifacts/data_ingestion/data.zip
